In [1]:
# contiene algoritmos de búsqueda implementado en AIMA
from search_mod import *

# algunas funciones auxiliares
from helpers_mod import *

In [12]:
psource(Problem)
psource(Node)
psource(expand)
psource(path_actions)
psource(path_states)
psource(PriorityQueue)
psource(breadth_first_search)
psource(depth_first_recursive_search)
psource(depth_limited_search)

psource (report)
psource (CountCalls)
psource (report_counts)

In [3]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h(self, node): return 0
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

In [4]:
class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [5]:
# Ejemplos de creación de instancias del problema del 8-puzzle
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

# resolver una instancia particular con una estrategia de búsqueda concreta
# retorna la solución, como una instancia de Node
print(f'Resolver el problema {e1} con Búsqueda en Anchura')
sol = breadth_first_search(e1)
print ("Nodo solución: ", sol)

Resolver el problema EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8), (1, 2, 3, 4, 5, 6, 7, 8, 0)) con Búsqueda en Anchura
Nodo solución:  <(1, 2, 3, 4, 5, 6, 7, 8, 0)>


In [6]:
# e imprimir informacion relevante utilizando los atributos y funciones siguientes
print (f'Coste del camino: {sol.path_cost:d}')
print (f'Estado solución: {sol.state:}')
print (f'Estado solución (tablero):')
print (board8(sol.state))
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))

# lista de estados formateado
for s in path_states(sol):
    print(board8(s))

Coste del camino: 23
Estado solución: (1, 2, 3, 4, 5, 6, 7, 8, 0)
Estado solución (tablero):
1 2 3
4 5 6
7 8 _

Lista de acciones: [4, 7, 6, 3, 4, 7, 8, 5, 4, 3, 6, 7, 4, 1, 2, 5, 8, 7, 6, 3, 4, 5, 8]
Numero de acciones en el camino: 23
Lista de estados:
[(1, 4, 2, 0, 7, 5, 3, 6, 8), (1, 4, 2, 7, 0, 5, 3, 6, 8), (1, 4, 2, 7, 6, 5, 3, 0, 8), (1, 4, 2, 7, 6, 5, 0, 3, 8), (1, 4, 2, 0, 6, 5, 7, 3, 8), (1, 4, 2, 6, 0, 5, 7, 3, 8), (1, 4, 2, 6, 3, 5, 7, 0, 8), (1, 4, 2, 6, 3, 5, 7, 8, 0), (1, 4, 2, 6, 3, 0, 7, 8, 5), (1, 4, 2, 6, 0, 3, 7, 8, 5), (1, 4, 2, 0, 6, 3, 7, 8, 5), (1, 4, 2, 7, 6, 3, 0, 8, 5), (1, 4, 2, 7, 6, 3, 8, 0, 5), (1, 4, 2, 7, 0, 3, 8, 6, 5), (1, 0, 2, 7, 4, 3, 8, 6, 5), (1, 2, 0, 7, 4, 3, 8, 6, 5), (1, 2, 3, 7, 4, 0, 8, 6, 5), (1, 2, 3, 7, 4, 5, 8, 6, 0), (1, 2, 3, 7, 4, 5, 8, 0, 6), (1, 2, 3, 7, 4, 5, 0, 8, 6), (1, 2, 3, 0, 4, 5, 7, 8, 6), (1, 2, 3, 4, 0, 5, 7, 8, 6), (1, 2, 3, 4, 5, 0, 7, 8, 6), (1, 2, 3, 4, 5, 6, 7, 8, 0)]
1 4 2
_ 7 5
3 6 8

1 4 2
7 _ 5
3 6 8

1 4 2
7 6 

In [7]:
psource (Localizaciones)

In [8]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print (g2.distancia(0,1))

55.88273580792048
119.30959564041359


In [9]:
class ProblemaViajanteComercio(Problem):
    """ Problema del Viajante de Comercio para 8 ciudades """
    
    def __init__(self, initial, localizaciones):
        super().__init__(initial)
        self.localizaciones = localizaciones
        self.nciudades = localizaciones.nciudades
    
    def actions(self, state):
        """ Devuelve las ciudades no visitadas como posibles acciones. """
        visitadas = set(state)
        return [ciudad for ciudad in range(self.nciudades) if ciudad not in visitadas]
    
    def result(self, state, action):
        """ Devuelve un nuevo estado añadiendo la ciudad visitada. """
        return state + (action,)
    
    def is_goal(self, state):
        """ El estado objetivo es cuando se han visitado todas las ciudades y volvemos al inicio. """
        return len(state) == self.nciudades and len(set(state)) == self.nciudades
    
    def action_cost(self, s, action, s1):
        """ Devuelve el costo de la distancia entre las ciudades """
        return self.localizaciones.distancia(s[-1], action) if s else 0
    
    def h(self, node):
        """ Heurística nula para búsqueda no informada. """
        return 0

# Función para resolver el problema con distintas estrategias
def resolver_problema(problema):
    estrategias = {
        "BFS": breadth_first_search,
        "DFS": depth_first_recursive_search,
        "Depth limited": depth_limited_search
    }
    
    resultados = []
    for nombre, estrategia in estrategias.items():
        solucion = estrategia(problema)
        nodos_expandidos = len(solucion) if solucion else "Sin solución"
        costo_camino = solucion.path_cost if solucion else "Sin solución"
        resultados.append((nombre, nodos_expandidos, costo_camino))
    
    return resultados

# Código para generar la tabla de resultados
def mostrar_resultados(resultados):
    print("\nEstrategia      | Nodos Expandidos | Costo del Camino")
    print("-" * 55)
    for estrategia, nodos, costo in resultados:
        print(f"{estrategia:15} | {nodos:16} | {costo}")

In [10]:
# Celda asociada
# Cargar las ubicaciones desde el archivo
localizaciones = Localizaciones('./data/grafo8cidades.txt')

# Estado inicial: comenzamos en la ciudad 0
estado_inicial = (0,)

# Crear el problema del Viajante de Comercio
problema = ProblemaViajanteComercio(estado_inicial, localizaciones)

# Resolver el problema usando distintas estrategias de búsqueda
resultados = resolver_problema(problema)

# Mostrar los resultados en una tabla
mostrar_resultados(resultados)


Estrategia      | Nodos Expandidos | Costo del Camino
-------------------------------------------------------
BFS             |                7 | 331.29286014394575
DFS             |                7 | 331.29286014394575
Depth limited   |                7 | 325.78722595962773
